TODO!
Setup data for a google sheets output
setup data for a twitter sentiment grab
graph all results

In [1]:
%config IPCompleter.greedy=True

In [2]:
import os, sys, collections, time, datetime
from IPython.display import Image as IpyImage
from PIL import Image
import requests
from io import BytesIO
# cryptostuffs
import ccxt
import cryptocompare as cc
# my stuffs
import options
import printer # i guess this example doesnt *need* this...

In [ ]:
# Dataset holders for infos
_dataset        = collections.defaultdict(dict)
betterset       = collections.defaultdict(dict)

In [ ]:
# Add into this list the API keys for your exchanges
P("Setting up Exchanges: One at a time.")
get_exchanges = ['binance', 
                 'poloniex', 
                 'cryptopia',
                 'hitbtc',
                 'bitmex',
                 'ccex',
                 'bittrex']

In [ ]:
def setup_exchange(this_exchange="binance", dataset=None):
    """
    This should go through each exchange available all at once and
    assign the everything to the data structure properly.
    """
    dataset[this_exchange]['ex_id'] = this_exchange
    exchange_connect = eval("ccxt.{}()".format(this_exchange))
    dataset[this_exchange]['app'] = exchange_connect
    exchange_connect.apiKey = eval("config.{}_api".format(this_exchange))
    exchange_connect.secret = eval("config.{}_secret".format(this_exchange))
    balances = {}
    try:
        balance = exchange_connect.fetch_balance()
    except:
        print("{} is Offline".format(this_exchange))
        return None
    for x in balance['total']:
        if float(balance['total'][x]) > 0:
            balances[x] = balance['total'][x]
            try:
                dataset['coins'][x] += '{}, '.format(this_exchange)
            except:
                dataset['coins'][x] = '{}, '.format(this_exchange)
    dataset[this_exchange]['balances'] = balances
    # get the ticker
    try:
        ticker = exchange_connect.fetch_tickers()
        dataset[this_exchange]['ticker'] = ticker
    except Exception as e:
        dataset[this_exchange]['ticker'] = None     
    return exchange_connect

In [ ]:
# Copy dummy.yaml to config.yaml | OR | add in your custom path to the dummy.yaml
# AND add in your API keys
config = options.Options("../../access/access_codes.yaml")
P = printer.Printer(config).no_size_printer
msg = "| AG INDEX | {}".format(datetime.datetime.now())
P(msg)
if False:
    # print some goods
    P("Config")
    print(config)

In [ ]:
# Get Master Balance
def get_balances(dataset):
    """Create Subset of Master Balance."""
    balances = {}
    for x in dataset:
        if 'coins' not in x:
            # print("Checking Balance on {}".format(str(x).upper()))
            try:
                for i in dataset[x]['balances']:
                    try:
                        balances[i] += float(dataset[x]['balances'][i])
                    except:
                        balances[i] = float(dataset[x]['balances'][i])
            except:
                print("Exchange: {} shows no balance!".format(x))
    return balances

In [ ]:
def get_imgs(dataset):
    url = base_url + x['BTC']['ImageUrl']
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    IpyImage(img)
    return True

In [ ]:
# get Basic coin data for all coins
if False:
    all_coin_data   = cc.get_coin_list(format=False)

In [ ]:
# binance is not working...
if True:
    my_exchanges_ = []
    for i in get_exchanges:
        x = setup_exchange(i, _dataset)
        my_exchanges_.append(x)
    print(my_exchanges_)

In [ ]:
my_balances = get_balances(_dataset)
print(my_balances)

In [ ]:
my_coin_data    = cc.get_price([coin for coin in my_balances if not None], curr='USD', full=True)

In [ ]:
# GET TOTALS FROM cryptocompare.com
if False:
    total_usd = 0
    for coin in my_coin_data['DISPLAY']:
        exchanges = [i for i in _dataset['coins'][coin].split(',')]
        P(); P("{}".format(coin)); P()
        if False:
            for stat in my_coin_data['DISPLAY'][coin]['USD']:
                P("{}: {}".format(str(stat).lower(), my_coin_data['DISPLAY'][coin]['USD'][stat]))
        data = my_coin_data['DISPLAY'][coin]['USD']
        bal = my_balances[coin]
        cur_price = my_coin_data['RAW'][coin]['USD']['price'.upper()]
        vol24 = my_coin_data['RAW'][coin]['USD']['volume24hour'.upper()]
        change_pct = my_coin_data['RAW'][coin]['USD']['changepct24hour'.upper()]
        usd_value = float(bal) * float(cur_price)
        P("Exchanges: {}".format(exchanges[:-1]))
        P("Balance: {:.8f}".format(bal))
        P("Current Price: {:.8f}".format(cur_price))
        # P("Volume 24: {:.2f}".format(vol24))
        # P("Change 24: {:.2f}".format(change_pct))
        P("Current USD Value: {:.2f}".format(usd_value)) 
        total_usd += usd_value
    P()
    P("Total USD(WRONG!): {:.2f}".format(total_usd)); P()

In [ ]:
# list coins and exchanges
if False:
    for coin in my_balances:
        exchanges_ = []
        for i in _dataset['coins'][coin].split(','):
            exchanges_.append(i)
        P("{} is traded on these Exchanges {}".format(coin, exchanges_[:-1]))

In [ ]:
# GET THE TOTALS from the tickers
for exchange in reversed(my_exchanges_):
    print("Checking coins against exchange:", exchange.id)
    for coin in my_balances: 
        if coin is not None:
            betterset[coin]['bal'] = my_balances[coin]
            try:
                # if we have good data on this coin... just move on for now...
                if betterset[coin]['found']:
                    print("already have coin: ", coin)
                    pass;
            except:
                if not 'BTC' in coin:
                    pair = '{}/BTC'.format(coin)
                else:
                    pair = '{}/USDT'.format(coin)
                # Search the Local Directory First then search invidually
                try:
                    for pairs in _dataset[exchange]['ticker']:
                        if pair in pairs:
                            subset = _dataset[exchange]['ticker']

                except:
                    # Not Found in Local Directory
                    print("Looking up coin: {}, for pair {} on exchange {}".format(coin, pair, exchange.id))
                    try:
                        subset = exchange.fetchTicker(pair)
                        time.sleep(2)
                    except:
                        pass;
                #//////////////////////////////////////////////////////
                try:
                    if subset:
                        betterset[coin]['pair'] = pair
                        betterset[coin]['high'] = subset['high']
                        betterset[coin]['low'] = subset['low']
                        betterset[coin]['last'] = subset['last']
                        betterset[coin]['change'] = subset['change']
                        betterset[coin]['ex_id'] = exchange.id
                        betterset[coin]['found'] = True
                        print("{} Found!".format(coin))
                        subset = None
                except:
                    pass;

In [ ]:
# LETS LOOK THROUGH OUR CAREFULLY PARSED DATA!!
btc_price = betterset['BTC']['last']
total_btc_val = 0
total_usd_val = 0
for i in sorted(betterset):
    if 'BTC' not in i:
        try:
            P(i)
            P('Balance: {:.8f}'.format(betterset[i]['bal']))
            P('Current BTC Price: {:.8f}'.format(betterset[i]['last']))
            cur_btc_val = float(betterset[i]['bal']) * float(betterset[i]['last'])
            total_btc_val += cur_btc_val
            P('Current {} Balance Value: {:.8f}'.format(betterset[i]['pair'][-3:], cur_btc_val))
        except:
            pass
P()
P('Total BTC Value: {:.8f}'.format(total_btc_val))
total_usd_val = float(btc_price) * total_btc_val
P('Total USD Value: {:.2f} $'.format(total_usd_val))
P()